# Manual de Creación de Aplicación Django con Formulario y Base de Datos PostgreSQL

### **1. Crear y activar un entorno virtual**

El entorno virtual permite aislar las dependencias del proyecto.

In [ ]:
python -m venv venv

In [ ]:
.\venv\Scripts\activate

---

### **2. Instalar las dependencias necesarias**

Instalamos `Django`, `psycopg2` para PostgreSQL y `djangorestframework`.

In [ ]:
pip install Django psycopg2 djangorestframework

---

### **3. Crear un nuevo proyecto Django**

In [ ]:
django-admin startproject user_project .

---

### **4. Configurar la conexión a PostgreSQL en `settings.py`**

In [ ]:
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': 'nombre_de_tu_base_de_datos',
        'USER': 'nombre_de_usuario',
        'PASSWORD': 'tu_contraseña',
        'HOST': 'localhost',
        'PORT': '5432',
    }
}

---

### **5. Crear una nueva aplicación**

In [ ]:
django-admin startapp user_app

Modifica `INSTALLED_APPS` en `settings.py` para incluir la aplicación y Django REST Framework.

In [ ]:
INSTALLED_APPS = [
    ..., 
    'user_app',
    'rest_framework',
]

---

### **6. Definir el modelo en `models.py`**

In [ ]:
from django.db import models

class UserProfile(models.Model):
    full_name = models.CharField(max_length=100)
    email = models.EmailField(unique=True)
    age = models.PositiveIntegerField()
    created_at = models.DateTimeField(auto_now_add=True)

---

### **7. Crear y aplicar las migraciones**

In [ ]:
python manage.py makemigrations

In [ ]:
python manage.py migrate

---

### **8. Registrar el modelo en `admin.py`**

In [ ]:
from django.contrib import admin
from .models import UserProfile

admin.site.register(UserProfile)

---

### **9. Crear el formulario en `forms.py`**

In [ ]:
from django import forms
from .models import UserProfile

class UserProfileForm(forms.ModelForm):
    class Meta:
        model = UserProfile
        fields = ['full_name', 'email', 'age']

---

### **10. Configurar la vista en `views.py`**

In [ ]:
from django.shortcuts import render, redirect
from .forms import UserProfileForm

def user_form_view(request):
    if request.method == 'POST':
        form = UserProfileForm(request.POST)
        if form.is_valid():
            form.save()
            return redirect('user-success')
    else:
        form = UserProfileForm()
    return render(request, 'user_form.html', {'form': form})

---

### **11. Crear la plantilla HTML en `templates/user_form.html`**

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <title>User Form</title>
</head>
<body>
    <h1>User Registration</h1>
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit">Submit</button>
    </form>
</body>
</html>

---

### **12. Definir las rutas en `urls.py` de la aplicación**

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path('form/', views.user_form_view, name='user-form'),
    path('success/', views.user_form_view, name='user-success'),
]

---

### **13. Configurar las rutas principales en `urls.py` del proyecto**

In [ ]:
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('api/v1/', include('user_app.urls')),
]

---

### **14. Configurar la vista API en `views.py`**

In [ ]:
from rest_framework import viewsets
from .models import UserProfile
from .serializers import UserProfileSerializer

class UserProfileViewSet(viewsets.ModelViewSet):
    queryset = UserProfile.objects.all()
    serializer_class = UserProfileSerializer

---

### **15. Crear el serializador en `serializers.py`**

In [ ]:
from rest_framework import serializers
from .models import UserProfile

class UserProfileSerializer(serializers.ModelSerializer):
    class Meta:
        model = UserProfile
        fields = '__all__'

---

### **16. Configurar las rutas de la API en `urls.py`**

In [ ]:
from rest_framework import routers
from .views import UserProfileViewSet

router = routers.DefaultRouter()
router.register(r'users', UserProfileViewSet)

urlpatterns += router.urls

---

### **17. Ejecutar el servidor**

In [ ]:
python manage.py runserver

---

### **18. Acceder a la aplicación**

- **Formulario web:** `http://127.0.0.1:8000/api/v1/form/`

- **Panel de administración:** `http://127.0.0.1:8000/admin/`

- **Endpoint REST:** `http://127.0.0.1:8000/api/v1/users/`